In [84]:
import pandas as pd
from dominate.tags import *
from dominate.util import raw
from bs4.dammit import EntitySubstitution
import os.path

In [66]:
papers = pd.read_csv('data.csv', encoding='UTF-8')
papers

,ID,Count,Timestamp,Email,Name,Title,Authors,Acronym,Workshop,Track,Workshop_ID,Please submit your camera ready version paper (.pdf),Please submit your signed author agreement files (.pdf)
0,CSEDM_1,1,6/28/2021 21:12,mfowler5@illinois.edu,Max Fowler,How productive are homework and elective pract...,"Max Fowler, Binglin Chen, Matthew West, Craig ...",CSEDM,5th Educational Data Mining in Computer Scienc...,Full Paper,CSEDM,https://drive.google.com/open?id=13bP8sLANoNDG...,https://drive.google.com/open?id=1efIq4pV8T2ge...
1,CSEDM_2,2,6/30/2021 8:40,jdpinto2@illinois.edu,Juan Pinto,Investigating Elements of Student Persistence ...,"Juan D. Pinto, Yingbin Zhang, Luc Paquette, Ay...",CSEDM,5th Educational Data Mining in Computer Scienc...,Full Paper,CSEDM,https://drive.google.com/open?id=16EntUbYtk2A0...,https://drive.google.com/open?id=10cXdkDGw2h7T...
2,CSEDM_3,3,6/28/2021 8:35,benjamin.paassen@hu-berlin.de,Benjamin Paaßen,Next Steps for Next-step Hints: Lessons Learne...,"Benjamin Paaßen, Jessica McBroom, Bryn Jeffrie...",CSEDM,5th Educational Data Mining in Computer Scienc...,Full Paper,CSEDM,https://drive.google.com/open?id=1Zjbn_ZapgcOO...,https://drive.google.com/open?id=1odpnePZof41Q...
3,CSEDM_4,4,7/12/2021 21:40,Mohsen.Dorodchi@uncc.edu,Mohsen Dorodchi,Clustering Students' Short Text Reflections: A...,"Mohsen Dorodchi, Alexandria Benedict, Erfan Al...",CSEDM,5th Educational Data Mining in Computer Scienc...,Full Paper,CSEDM,https://drive.google.com/open?id=1_HNIgj75-iI8...,https://drive.google.com/open?id=1BiFJHA3z7weR...
4,CSEDM_5,5,6/30/2021 13:53,wwang33@ncsu.edu,Wengran Wang,Execution-Trace-Based Feature Engineering To E...,"Wengran Wang, Gordon Fraser, Tiffany Barnes, C...",CSEDM,5th Educational Data Mining in Computer Scienc...,Full Paper,CSEDM,https://drive.google.com/open?id=1_Ko4psbtfvqg...,https://drive.google.com/open?id=1lVOglu7sM1tZ...
5,CSEDM_6,6,6/29/2021 8:40,cristina@gsic.uva.es,Cristina Villa Torrano,Dataset on an online collaborative learning si...,"Cristina Villa-Torrano, Pankaj Chejara, Juan I...",CSEDM,5th Educational Data Mining in Computer Scienc...,Abstract,CSEDM,https://drive.google.com/open?id=1rDIxot5NZrQn...,https://drive.google.com/open?id=1CkzCWa6el16j...
6,CSEDM_7,7,6/29/2021 13:20,profmdn@vt.edu,Mostafa Mohammed,Clickstream Data from a Formal Languages eText...,"Mostafa Mohammed, and Clifford Shaffer",CSEDM,5th Educational Data Mining in Computer Scienc...,Abstract,CSEDM,https://drive.google.com/open?id=13Rhe3RKgwVYt...,https://drive.google.com/open?id=1HQH9bxGaP7a5...
7,CSEDM_8,8,6/27/2021 11:36,aolney@memphis.edu,Andrew M. Olney,"JupyterLab Extensions for Blocks Programming, ...","Andrew M. Olney, Scott D. Fleming",CSEDM,5th Educational Data Mining in Computer Scienc...,Abstract,CSEDM,https://drive.google.com/open?id=1efilFZYOej8R...,https://drive.google.com/open?id=1N6OEsE0jRpKC...
8,CSEDM_9,9,6/25/2021 12:23,awh4kc@vt.edu,Alexander Hicks,Containerizing an eTextbook Infrastructure,"Alexander Hicks, Clifford A. Shaffer",CSEDM,5th Educational Data Mining in Computer Scienc...,Abstract,CSEDM,https://drive.google.com/open?id=1Cm-2zUOC-suG...,https://drive.google.com/open?id=1n1dq_pNkURO8...
9,CSEDM_10,10,6/29/2021 10:53,barbarer@umich.edu,Barbara Jane Ericson,Comparing Ebook Student Interactions With Test...,"Hisamitsu Maeda, Barbara Ericson, Paramveer Dh...",CSEDM,5th Educational Data Mining in Computer Scienc...,Work in Progress,CSEDM,https://drive.google.com/open?id=1W-v5iecHhtV-...,https://drive.google.com/open?id=12hY7Z9eXegP7...


In [88]:
organizers = {
    'CSEDM': 'Bita Akram (NC State University), Thomas Price (NC State University), Yang Shi (NC State University), Peter Brusilovsky (University of Pittsburgh), Sharon Hsiao (Arizona State University)',
    'LDI': 'Vasile Rus (University of Memphis), Stephen E. Fancsali (Carnegie Learning)',
    'UGR': 'Ryan Baker (University of Pennsylvania), Caitlin Mills (University of New Hampshire), Ulrich Boser (The Learning Agency)',
    'PA': 'Nirmal Patel (Playpower Labs	), Derek Lomas (Delft University of Technology), Tirth Shah (Playpower Labs)',
}

In [89]:
toc = div(h2('Table of Contents'), cls="CEURTOC")
toc_list = ul()
toc.add(toc_list)

workshop_index = 1
for workshop_id in papers['Workshop_ID'].unique():
    title = papers[papers['Workshop_ID'] == workshop_id]['Workshop'].iloc[0]
    title = f'Workshop {workshop_index}: {title}'
    toc_list.add(li(a(title, href=f'#{workshop_id}'), id=f'ws{workshop_index}'))
    workshop_index += 1


preface_default = """
Summary: There were <span class="CEURSUBMITTEDPAPERS">49</span> papers submitted for peer-review to this workshop. Out of these, 
<span class="CEURACCEPTEDPAPERS">20</span> papers were accepted for this volume, 
<span class="CEURACCEPTEDREGULARPAPERS">12</span> as regular papers and
<span class="CEURACCEPTEDSHORTPAPERS">8</span> as short papers.
"""

escaper = EntitySubstitution()
def esc(html):
    return html # raw(escaper.substitute_html(html))

def confirm_file(file):
    if not os.path.isfile('Proceedings/' + file):
        print('Missing file: ' + file)
    return file

session_count = 1
for workshop in papers['Workshop'].unique():
    toc.add(hr())
    
    ws_id = papers[papers['Workshop'] == workshop]['Workshop_ID'].iloc[0]
    toc.add(h3(span('Workshop ' + str(session_count) + ': ' + workshop, cls="CEURTITLE"), id=ws_id))
    session_count += 1
    
    toc.add('Organizers: ')
    toc.add(raw(organizers[ws_id]))
    toc.add(br())
    
    preface = li(id="xpreface")
    toc.add(ul(preface))
    preface_file = confirm_file(f'x{ws_id}_preface.pdf')
    preface.add(a('Preface', href=preface_file))
    preface.add(br())
#     preface.add(preface_default)
#     toc.add(br())
    
    wks = ul()
    toc.add(wks)
    
    workshop_papers = papers[papers['Workshop'] == workshop]
    show_types = len(workshop_papers['Track'].unique()) > 1
    
    for index, row in workshop_papers.iterrows():
        paper = li(id=row['ID'])
        wks.add(paper)
        title = row['Title']
        if show_types:
            title += f' ({row["Track"].strip()})'
        paper.add(a(span(esc(title), cls='CEURTITLE'), href=confirm_file(row['ID'] + '.pdf')))
        paper.add(br())
        # paper.add(span(cls='CEURTITLE'))
        authors = row['Authors'].split(', ')
        authors = map(lambda author: author[4:] if author.startswith('and ') else author, authors)
        for author in authors:
            paper.add(span(esc(author), cls='CEURAUTHOR'))
        wks.add(br())
        
# print(toc)
with open('index.html', 'r') as f:
    base = f.read()
    base = base.replace('[INSERT TOC]', str(toc))

    with open('Proceedings/index.html', 'wb') as w:
        w.write(base.encode('utf-8'))
